### How to run your code?

Running the code simply is running the cells in order on this notebook.

The very last cell contains 

```runGame(false, true) # (monte_carlo_enabled, random_play_enabled, board_width, board_height)```

Set the first argument to true to enable MonteCarlo INSTEAD of Minimax. Use the second argument to enable random play. You can also optionally set the board heigh and board width (starts as 6x7).



### Question Answers

**1. (10 points) Provide an upper bound for the number of positions in
Connect Four on a 7×6 board. Explain how you derived your answer:**

There are are 6x7 = 42 different "squares" on the board. Each place has the option of 2 "pieces" in it, or blank. It is a permutation of all of the squares and possible states: for the first 3 squares, it would be 3x3x3 = 27 different permutations. Therefore, the upper bound is 3^42.

This upper bound isn't quite accurate though. Players must alternate turns, significantly cutting down the state space. There are many impossible states with the previous estimate taken into account. The most impactful one is that there cannot be blank squares under pieces. There are also positions that are impossible to achieve after a players achieves 4 in a row (example: the bottom row having 4 in a row, and second row completely full as well).

So I leave this 3^42 as a  crude upper bound estimate that could be improved upon.

**2. (15 points) Implement the game Connect Four in Julia. Implement
the board, positions in the game, two players, moving in turns, and
a test that determined whether the game has been won. Your implementation must detect illegal moves and raise an error when a move
is illegal. The default board in Connect Four has a size of 7 × 6:**

Implemented in the cells below. The game will end when a winning position is reached and will print the winner. Turns are taken, with the human player entering input manually and the AI making moves afterwards automatically. Illegal moves raise proper errors and displays messages to the user, allowing them to fix their mistakes.

**3. (25 points) Implement an intelligent agent to play Connect Four using
the Minimax algorithm. Use a heuristic value function to terminate
your search early. How many plys can you look ahead? What is the
branching factor of your game tree?**

Minimax is implemented in the secion "Minimax". Minimax can be run (without random play) by changing the last cell to:

```runGame(false, false)``` 

Without random moves enabled, Minimax can look 6 plays ahead in a tolerable time (less than 10 seconds). Anymore than that and it isn't very fun to play. I usually leave it at 4 plays ahead and it makes a move within a second.

Enabling random moves makes minimax take quite a bit more time.

The branching factor is the number of columns, which by default is 7.

**(15 points) Consider a modified version of the Connect Four game in
which both players have one additional type of move they can play:
randomly drop one stone in an open column and force the next player
to also randomly drop one stone in an open column. Modify your
algorithm to play this modified version of Connect Four:**

Random is enabled by setting the second parameter in runGame:

```runGame(false, true)``` 

-1 is what you should enter as input to make a random move (instructions are printed when you start a game).

**(35 points) Modify your agent to use Monte Carlo Tree Search instead
of the Minimax algorithm. You can use a random playout or design
a different strategy.**

Tree search is implemented in the "MonteCarlo" section, using random playout.


### Email Questions

**What are the assumptions you made (your heuristic, your design decisions and justifications to those decisions, and the answers to the question)?**

**Design decisions includes number of plays ahead, the way you designed your random move and the exploration constant and the number of simulations in MCST?**

MINIMAX algorithm heuristic works as follows:

- Check the lookup table: 


    "win" => 1000,
    "lose" => -1000,
    "draw" => 0,
    3 connected pieces => 3*3 (9 points), 
    2 connected pieces => 2*2 (3 points),
    1 connected piece =>  1*1 (1 point)
    
    
- A player's score is calculated by scanning the entire board and adding the number of connected pieces.
    

- Additionally, the number of blanks surrounding a piece are taken into account. For each piece on the board, it adds the following score:


    "3 blanks" => 3/10 of a point
    "2 blanks" => 2/10 of a point
    "1 blank"  => 1/10 of a point
    
    
- This essentially works as a tie breaker for moves that make the same number of connected pieces on the board. Having more blanks (such as playing in the center of the board at the beginning) is better.
   
   
- Of course, winning (4 or more connected pieces in the implementation) is worth a lot of points.

   
- The board is scanned by the algorithm and points are assessed for both players. In order to calculate the current player's score, you take the difference of the 2 players.


*In the coded implementation, the score is always computed considering the second player as the Max player. The first player tries to minimize their score, and the second player tries to maximize their score.

Justification:
This strategy seems intuitively like a fair way to score. The more connected pieces you have, the closer you are to winning. Furthermore, 2 connected pieces is better than 2 separated pieces, so it is weighted more. The real justification is that this algorithm beats me every time I play!


-----

MONTECARLO is as follows:

Use the following policy to check for "common sense" play: 

- If you can win with the next move, make it.
- If the opponent is going to win after your next move, don't make that move!

After this selection doesn't apply, then move to expansion and simulation. 


Expnasion & Simulation: Simulation is limited to 3 seconds, which simulates over 40,000 unique states. Random Play from both players initially when there are no stats available. If there are stats available for states, it will choose states with a higher win rate to sample statistcally more (weighted proportinally) over other states. There is a certain probability (1 - highest_win_rate) to randomly select a new unexplored state instead as well.

I took the design decision to not limit the branching factor in this montecarlo algorithm. The reason being that 7 moves is not very many, and that way you don't miss any obvious moves.

I think it is an intuitive design for monte-carlo to work. Use random moves (especially when the branching factor is so small) to choose the next states. If stats are available about the win rates, then utilize this information to explore those states more frequently (statistically determined).

MonteCarlo also plays quite well, and I have a tough time beating it (depending on how much time is given to the algorithm).


In [4]:
include("./game.jl")
include("./board.jl")
using .game
using Random
using Statistics

In [14]:
# Change these variables if desired
MONTO_CARLO_DURATION = 4    # Seconds
MINIMAX_MOVES_AHEAD = 4

4

### Helper functions

In [6]:
function generate_possible_games(mygame::game.Game)
    moves = game.available_moves(mygame)
    new_games = game.Game[]

    # Make all new possible moves
    for move in moves
        new_game = deepcopy(mygame)
        game.move(new_game, move)
        push!(new_games, new_game)
    end 
    return new_games
end


function randomMove(mygame::game.Game)
    moves = game.available_moves(mygame)
    choice = rand(1:length(moves))
    move = moves[choice]
    return move 
end


randomMove (generic function with 1 method)

# Minimax

In [7]:
RANDOM_MOVE = -1


# Values for minimax
SCORES = Dict{Union{String, Integer}, Integer}(
    "win" => 1000,
    "lose" => -1000,
    "draw" => 0,

    # For connecting multiple pieces
    4 => 1000,
    3 => 3,
    2 => 2,
    1 => 1,
)

function minimax(mygame::game.Game, random_play)
    cur_depth = 0
    max_depth = MINIMAX_MOVES_AHEAD
    cost, move = mm(cur_depth, max_depth, mygame, true, random_play)
    return move
end


# Returns (score, move) tuple
function mm(cur_depth::Int64, max_depth::Int64, mygame::game.Game, hero=true, random_play=false)
    
    # Hero is the max_player
    if (cur_depth > max_depth) | game.winner(mygame) | game.finished(mygame)
        # Always considered 'max' for player 1. Sooner wins are better.
        return evaluation(mygame) - cur_depth, mygame.lastMove
    end

    # Recursive
    choices = []
    for gm in generate_possible_games(mygame)
        score, _ = mm(cur_depth + 1, max_depth, gm, !hero)
        push!(choices, (score=score, move=gm.lastMove.y))
    end

    # Random Move
    if random_play
        random_scores = []
        for gm in generate_possible_games(mygame)
            winner = game.winner(gm)                        # max_player won by making this move
            for g in generate_possible_games(mygame)
                if winner                                   # Won/lost on the last move
                    score = evaluation(mygame) - cur_depth
                else      
                    score, _ = mm(cur_depth + 2, max_depth, gm, hero)
                end
                push!(random_scores, score)
            end
        end
        random_score = mean(random_scores)   # Take the average score
        push!(choices, (score=random_score, move=RANDOM_MOVE))
    end

    
    shuffle!(choices)   # Make different moves that are the same value
    fn(x) = x.score  # Gets the cost from the returned (cost, move) tuple
    findfn = hero ? findmax : findmin
    
    val, choice_index = findfn(map(fn, choices)) 
    
    best_move = choices[choice_index].move
    return val, best_move
end


# Minimax Evaluation Function, hero_index set to 2 because the player 2 is always the MAX player
function evaluation(mygame::game.Game, hero_index=2)  
    if game.finished(mygame) # Tie
        return 0
    end

    state = mygame.board.state
    fns = [game._checkVertical, game._checkDiagonal1, game._checkDiagonal2, game._checkHorizontal] 

    # Evaluate score for both players, then subtract
    players = mygame.players
    row, col, = size(state)

    # Scores for each player
    scores = Dict{String, Float64}()
    for player in players
        scores[player.token] = 0
    end

    # Loop through board
    for i = 1:row; for j = 1:col
        token = state[i, j]
        
        if token == gameboard.FILLER_CHAR 
            continue
        end

        # Otherwise, it's a player's character, so calculate SCORES
        # Currently, calculating connected pieces multiple times but that should be okay
        # A three connection will be calculated three times, two connection two times, 1 connection 1 time
        # So 3*3, 2*2, 1*1 weighting to the scores
        score = 0
        for fn in fns
            connected = fn(mygame, i, j)
            connected = min(connected, 4)    # connection of 5, 6, 7, etc don't mean anything
            score += SCORES[connected]
        end
        score -= length(fns) - 1             # Prevent counting the same [i, j] multiple times 

        # check_blanks is essiantly a tie breaker for moves - it's always better to have more connected pieces though
        score += game._checkBlanks(mygame, i, j) / 10    
        scores[token] += score
    end; end

    player_scores = Float64[]   # Scores for each player, set at the same index of the player
    for player in players
        push!(player_scores, scores[player.token])
    end

    hero = player_scores[hero_index] 
    enemy = sum(player_scores) - hero
    return hero - enemy
end

evaluation (generic function with 2 methods)

# Monte Carlo

In [8]:
function expansion(mygame, states)
    # Expand statistically and proportinally to the next states. Returns the move to make
    mygame = deepcopy(mygame)
    moves = []
    unexplored = []
    
    for gm in generate_possible_games(mygame)
        stats = get(states, gm.board.state, 0)
        if stats == 0  # Unvisited state
            win_rate = -1
            push!(unexplored, gm.lastMove.y)
        else
            win_rate = stats.wins / (stats.wins + stats.losses)
            push!(moves, (win_rate=win_rate, move=gm.lastMove.y))
        end
    end
    
    # No stats on the current moves yet, so pick a random one 
    if length(moves) == 0
        return game.randomMove(mygame)
    end
    
    # Find max win rate
    max_win_rate = -1
    for (win_rate, move) in moves
        if win_rate > max_win_rate
           max_win_rate = win_rate 
        end
    end
    
    # Explore new state with probabliity 1 - max_win_rate
    if rand() > max_win_rate
        if length(unexplored) == 0
            return game.randomMove(mygame)
        end
        choice = rand(1:length(unexplored))
        move = unexplored[choice]
        return move          
    end
    
    # Otherwise, return new move probabilistically determined by weighted win rate of states
    # State with more wins will be explored more

    moves = sort(moves, by = x -> x.win_rate)  # Sort by win rate
    
    for (win_rate, move) in moves
       if rand() < win_rate
            return move
       end
    end
    return moves[length(moves)].move

end
    

expansion (generic function with 1 method)

In [9]:
WIN = 1
LOSE = -1
TIE = 0


struct MonteCarlo
    # Dictionary of {board.state: Stats} - access any state quickly, and return the stats associated with it
    # AKA quick access to the game tree
    states
    game       # Current game (initial state and root of the tree) 
end


# Utility function
function makeMonteCarlo(mygame)
    mc = MonteCarlo(Dict(), mygame)
    add_state(mygame, mc.states)
    return mc
end


mutable struct Stats
    game::Any            # Type game.Game: keep meta about the game that was playe
    wins::Integer  
    losses::Integer  
    ties::Integer    
    parents::Any  # Set of states: keep track parent states
    children::Any # Set of states: keep track of children states
end


# Add a new state to the tree
function add_state(mygame, states, parent=nothing)
    # If state doesn't exist yet, add it
    state = mygame.board.state
    stats = get(states, state, 0)

    if stats == 0  # Doesn't exist yet
        stats = Stats(deepcopy(mygame), 0, 0, 0, Set(), Set())
        states[mygame.board.state] = stats
    end

    if parent != nothing
        push!(states[parent].children, state)
        push!(stats.parents, parent)
    end
end


    
# Return the end state from a random simulation, and who is the winner (or tie)
function simulation(state, states)
    mygame = states[state].game
    mygame = deepcopy(mygame)
    turn = mygame.turn

    # EXPANSION
    #move = expansion(mygame, states)
    game.moveRandom(mygame)
    add_state(mygame, states, state)
    # END
    
    if game.winner(mygame)
        return turn, mygame.board.state
    elseif game.finished(mygame)        # Tie Game
        return 0, mygame.board.state
    else
        # SIMULATE UNTIL END STATE
        return simulation(mygame.board.state, states)
    end
end

# Backpropogate receursively up the tree given a finished state 
function backpropogation(state, states, outcome)
    # TIE, WIN, AND LOSS
    @assert -1 <= outcome <= 1

    for pstate in states[state].parents
        pstat = states[pstate]
        if outcome == WIN
            pstat.wins += 1
        elseif outcome == LOSE
            pstat.losses += 1
        elseif outcome == TIE
            pstat.ties += 1
        end

        # RECURSIVELY BACKPROP UP THE TREE
        backpropogation(pstate, states, outcome)
    end
end

# Run simulation recursively one time, and backpropogate to fill the stats
function simulate_and_backprop(mc::MonteCarlo)
    winner, end_state = simulation(mc.game.board.state, mc.states)

    if winner == mc.game.turn
        outcome = WIN
    elseif winner == game.nextTurn(mc.game)
        outcome = LOSE
    elseif winner == 0
        outcome = TIE
    else
        outcome = nothing
    end

    # UPDATE AND BACKPROP
    backpropogation(end_state, mc.states, outcome)
    return winner, end_state
end


# Make the game winning move if it's available
# AKA - don't be stupid
function policyMove(mc::MonteCarlo)
    curr_state = mc.game.board.state
    curr_stats = mc.states[curr_state]

    # Make the move to win
    for state in curr_stats.children
        stats = mc.states[state]
        if game.winner(stats.game)
            return stats.game.lastMove.y
        end
    end
    return nothing
end

# A policy that says which moves not to do
# Decides if 1 move in the future results in a win for the opponent, don't do it
# AKA - don't be stupid
function policyDontMove(mc::MonteCarlo)
    curr_state = mc.game.board.state
    curr_stats = mc.states[curr_state]

    # Don't make an immediately stupid move
    bad_moves = []
    for state1 in curr_stats.children
        stats1 = mc.states[state1] 
        for state2 in mc.states[state1].children
            stats = mc.states[state2]

            # If player lost the game, don't make the move
            if game.winner(stats.game)
                push!(bad_moves, stats1.game.lastMove.y)
            end
        end
    end
    return bad_moves
end


# Finds the next best move after a simulation has occured
function nextBestMove(mc::MonteCarlo, random_enabled)

    # SELECTION
    policy_move = policyMove(mc)

    if policy_move != nothing
        return policy_move
    end

    bad_moves = policyDontMove(mc)
    # END SELECTION

    curr_state = mc.game.board.state
    curr_stats = mc.states[curr_state]
    best_win_rate, best_move = -1, -1

    # LOOK AT STATS FROM SIMULATION 
    for state in curr_stats.children
        stats = mc.states[state]
        wins, losses = stats.wins, stats.losses
        win_rate = wins / (wins + losses)
        move = stats.game.lastMove.y

        if move in bad_moves
            continue
        elseif win_rate > best_win_rate
            best_win_rate = win_rate
            best_move = move  
        end 
    end
    
    # Return make-random-move if going to lose in all random simulations
    if best_win_rate == 0 && random_enabled
        return -1
    end
    
    return best_move
end

# Pass a game state, and do a simulation on it to find the best move
function monteCarloSimulation(mygame, random_enabled)
    mygame = deepcopy(mygame)
    mc = makeMonteCarlo(mygame)

    duration = MONTO_CARLO_DURATION # seconds
    start = finish = time()
    while finish - start < duration
        finish = time()
        simulate_and_backprop(mc)
    end 

    move = nextBestMove(mc, random_enabled)
    return move
end

monteCarloSimulation (generic function with 1 method)

# Run the game

In [10]:
STOP = "stop"

function makeAiMove(mygame::game.Game, random_enabled, monte_carlo=false)
    if monte_carlo
        move = monteCarloSimulation(mygame, random_enabled)
    else
        move = minimax(mygame, random_enabled)
    end
    return move
end

function runGame(use_monte_carlo=false::Bool, random_enabled=true::Bool, board_width=7, board_height=6)
    mygame = game.initializeGame("Human", "Computer", board_height, board_width)
    display(mygame.board.state)
    println()

    users_turn = true
    force_random_move = false    # Force random move for the next move
    
    println("Please enter the correct column to play a piece. Computer will play automatically. If random is enabled, enter '-1' to play a random move. To stop the game, enter 'stop'")
    
    while !game.winner(mygame) && !game.finished(mygame)
        if !force_random_move
            move = users_turn ? makeUserMove(mygame) : makeAiMove(mygame, random_enabled, use_monte_carlo)
            if move == STOP
                print("Game stopped by user")
                return
            end
        end

        if force_random_move || (move == RANDOM_MOVE) 
            move = game.randomMove(mygame)
            force_random_move = !force_random_move #  Change to true if the next player needs to make a random move, or false if it's the second player's move
        end
        
        println("Moving: ", move)
        game.move(mygame, move)
        users_turn = !users_turn
        display(mygame.board.state)
        println()
    end

    if game.winner(mygame)
        msg = string(game.previousPlayer(mygame).name, " wins!")
    else
        msg = "It's a draw." 
    end
    println("Game finished! $msg")
end

function makeUserMove(mygame::game.Game)
    while true
        move = getUserInput()
        if move == RANDOM_MOVE
            return RANDOM_MOVE
        elseif move == STOP
            return STOP
        end
        try
            game.validate_move_row(mygame, move)
            return move
        catch error
            println(error)
        end
    end
    
end

function getUserInput()
    move = nothing
    while true
        print("Your move: ")
        try
            move = readline(stdin)
            if move == STOP
                return move
            end
            move = parse(Int64, move)
            break
        catch error
            println("Please enter a valid column number")
        end
    end
    return move
end

getUserInput (generic function with 1 method)

# Run Game

In [ ]:
runGame(true, false, 7, 6) # (monte_carlo_enabled, random_play_enabled, width, height)

6×7 Array{String,2}:
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"


Please enter the correct column to play a piece. Computer will play automatically. If random is enabled, enter '-1' to play a random move. To stop the game, enter 'stop'
Your move: stdin> 4


6×7 Array{String,2}:
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "X"  "-"  "-"  "-"

Moving: 4



6×7 Array{String,2}:
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "X"  "O"  "-"  "-"

...

Moving: 2

Your move: stdin> 6


6×7 Array{String,2}:
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "O"  "-"  "-"  "-"
 "-"  "-"  "-"  "O"  "-"  "-"  "-"
 "-"  "O"  "X"  "X"  "X"  "-"  "-"
 "-"  "X"  "O"  "X"  "O"  "X"  "-"

Moving: 6



6×7 Array{String,2}:
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "-"  "-"  "-"  "-"
 "-"  "-"  "-"  "O"  "-"  "-"  "-"
 "-"  "-"  "-"  "O"  "-"  "-"  "-"
 "-"  "O"  "X"  "X"  "X"  "O"  "-"
 "-"  "X"  "O"  "X"  "O"  "X"  "-"

Moving: 6

Your move: 